<a href="https://colab.research.google.com/github/ViennaRNA/RNAdeep/blob/main/learningcurves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount Drive
#TODO Change to wget
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
os.listdir()

In [ ]:
!cp -r drive/My\ Drive/data/interim/100000_length70 .
!cp drive/My\ Drive/models/*py .

In [ ]:
#imports
from lstm_models import blstm
from metrics import mcc,f1,sensitivity
from data_generators import DataSliceFromInterimFile
import numpy as np
import os
from tensorflow.keras.callbacks import CSVLogger,ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
train_sizes = [10,100,1000,5000,8000,10000,30000,50000,70000,80000]
layers = 1
neurons = 40
batch_size = 128
for size in train_sizes:
  csvname = "lstm_%i_%i_%i_%i.csv" % (batch_size,size,layers,neurons)
  csv_logger = CSVLogger(csvname, append=True, separator=';')
  directory = "100000_length70"
  m = blstm(layers,neurons)
  opt = Adam(learning_rate=0.01)
  m.compile(optimizer=opt,loss="binary_crossentropy", metrics=["acc",mcc,f1])
  train_generator = DataSliceFromInterimFile(batch_size,os.path.join(directory,"train"),size)
  val_generator = DataSliceFromInterimFile(batch_size,os.path.join(directory,"val"),size)
  checkpoint_filepath = "lstm_%i_%i_%i_%i" % (batch_size,size,layers,neurons)
  model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_mcc', mode='max', save_best_only=True)
  print(checkpoint_filepath)
  m.fit(x=train_generator, validation_data = val_generator,epochs=50,shuffle = True,verbose=1,callbacks=[csv_logger,model_checkpoint])

In [ ]:
!zip -r lstm_128_1_40_learningsurve.zip  lstm_128_*